# Surface Weigthing

As was shown previously, a player's perfromance in a given match is greatly influenced by the court surface used in a match. 

In [2]:
# Init
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, lit

spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

def init_df(): 
    return spark.read \
    .csv("./dataset/all_matches.csv", header=True)

In [19]:
from pyspark.sql.functions import col, expr, when, lit

def winrate_any_surface(players):
    df = init_df()
    df = df.select(["player_id", "player_victory"]) \
        .where(df['player_id'].isin(players)) \
        .withColumn("player_victory", \
              when(df["player_victory"] == 't', 1).otherwise(0)) \
        .withColumn('matches', lit(1)) \
        .groupBy(['player_id']) \
        .sum()
    return df.withColumn('winrate', df['sum(player_victory)'] / df['sum(matches)']) \
        .withColumnRenamed('sum(matches)', 'total_matches') \
        .select(['player_id', 'winrate', 'total_matches'])

def winrate_per_surface(players):
    winrate = winrate_any_surface(players)
    df = init_df()
    df = df.select(["player_id", "court_surface", "player_victory"]) \
        .where(df['player_id'].isin(players)) \
        .withColumn("player_victory", \
                  when(df["player_victory"] == 't', 1).otherwise(0)) \
        .withColumn('matches', lit(1)) \
        .groupBy(['player_id', 'court_surface']) \
        .sum() \
        .join(winrate, on='player_id')

    df = df.withColumn('surface_winrate', df['sum(player_victory)'] / df['sum(matches)'])
    return df.withColumn('surface_advantage', df['surface_winrate'] - df['winrate'])
players = ['roger-federer', 'rafael-nadal', 'novak-djokovic', 'andy-murray']

winrate_per_surface(['roger-federer', 'rafael-nadal']).show()

+-------------+-------------+-------------------+------------+------------------+-------------+------------------+--------------------+
|    player_id|court_surface|sum(player_victory)|sum(matches)|           winrate|total_matches|   surface_winrate|   surface_advantage|
+-------------+-------------+-------------------+------------+------------------+-------------+------------------+--------------------+
| rafael-nadal|         Clay|                459|         526|0.8223905723905723|         1188|0.8726235741444867| 0.05023300175391432|
| rafael-nadal|        Grass|                 83|         105|0.8223905723905723|         1188|0.7904761904761904|-0.03191438191438...|
| rafael-nadal|         Hard|                428|         547|0.8223905723905723|         1188|0.7824497257769653|-0.03994084661360...|
| rafael-nadal|       Carpet|                  7|          10|0.8223905723905723|         1188|               0.7|-0.12239057239057238|
|roger-federer|         Clay|                263